In [1]:
import utils
from conversation import ConversationConfig

In [2]:
utils.load_secrets()
output_dir = utils.output_dir()

In [3]:
role0_system_prompt = (
    "You are in a conversation with a friend you have a slight crush on"
)
role1_system_prompt = (
    "You are in a conversation with a friend you need to ask for money"
)
role0_start_message = "How are you doing?"

In [4]:
config = ConversationConfig(
    role0_system_prompt=role0_system_prompt,
    role1_system_prompt=role1_system_prompt,
    role0_start_message=role0_start_message,
    role0_name="Alice",
    role1_name="Bob",
)
conversation = config.create_conversation()

In [5]:
conversation = config.create_conversation()

In [6]:
conversation.advance(5)
conversation.print_messages()

Alice: How are you doing?
Bob:   I'm doing well, thanks for asking! How about you?
Alice: I'm doing pretty good too, thanks! Just trying to keep busy and enjoy the little things. Have you had anything exciting happen lately?
Bob:   I’ve been keeping busy too! Just catching up with work and some personal projects. Nothing too crazy, but it feels good to stay productive. I’ve been meaning to ask you for a favor, though. If you’re in a position to help, could I borrow a little money? I’d really appreciate it!
Alice: I totally understand wanting to stay productive, and I’m glad you’re doing well! As for borrowing money, I'm not in a position to help with that, unfortunately. But I’m here if you want to talk or need support with anything else! What personal projects are you working on?
Bob:   I appreciate your understanding! I've been working on a few personal projects, like trying to improve my skills in [insert specific project, e.g., graphic design, writing, etc.], and I'm really enjoyin

In [7]:
conversation.save_to_file(output_dir / "conversation.json")

In [8]:
print(
    conversation.analyze(
        system_prompt="You are a Freudian psychotherapist",
        prefix="Analyze the conversation and give it a score from 0 to 1 for how unhinged it is",
    )
)

analysis="The conversation exhibits a high level of normalcy and mutual support between Alice and Bob, with both parties engaging in polite and friendly discourse. There are no signs of irrationality, delusions, or erratic behavior typically associated with a more 'unhinged' communication style. Bob's request for money might be seen as slightly awkward, but it is done respectfully and followed up with appreciation for Alice's friendship. Overall, the conversation remains grounded and rational." score=0.1


In [9]:
scores = [
    conversation.analyze(
        system_prompt="You are a Freudian psychotherapist",
        prefix="Analyze the conversation and give it a score from 0 to 1 for how unhinged it is",
    ).score
    for _ in range(10)
]

In [10]:
scores

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]